In [ ]:
!pip install -qU langchain-core langchain-openai langchain_huggingface langchain-community langsmith

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
!pip install -qU langchain-docling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.6/507.6 kB 37.6 MB/s eta 0:00:00
   ━

In [ ]:
!pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 9.4 MB/s eta 0:00:00


In [ ]:
! pip install rank_bm25

In [ ]:
from google.colab import userdata
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from IPython.display import display, Markdown
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import LLMChain
from langchain_community.llms import HuggingFaceHub
from langchain.vectorstores import FAISS
from langchain import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import Qdrant
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.embeddings import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from pprint import pprint
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from transformers import AutoTokenizer
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
huggingfacehub_api_token = userdata.get('HF_TOKEN')

In [ ]:
Qdrant_api_key = userdata.get('api_key')

In [ ]:
llm = ChatHuggingFace(
  llm=HuggingFaceEndpoint(
      repo_id="Qwen/Qwen3-Coder-480B-A35B-Instruct",
      provider="together",
      huggingfacehub_api_token=huggingfacehub_api_token,
      task="conversational"
  )
)

# Load data

In [ ]:
url='/content/transcript-20210724 (2).pdf'
question="Return to me list for all courses in Main Specialization Requirements with his hours and grade "

In [ ]:
def prepare_data(url):
  FILE_PATH = url
  loader = DoclingLoader(file_path=FILE_PATH, export_type= ExportType.MARKDOWN).load()
  tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
  text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
      tokenizer, chunk_size=300, chunk_overlap=20
  )
  normal_chunks = text_splitter.create_documents([loader[0].model_dump()['page_content']], metadatas=[loader[0].model_dump()['metadata']])
  return normal_chunks

# Vector Database

In [ ]:
def Hybrid_search(normal_chunks):
    model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

    embedding_llm = HuggingFaceEmbeddings(model_name=model_name)
    qdrant_store = Qdrant.from_documents(
    documents=normal_chunks,
    embedding=embedding_llm,
    url="https://3464a78e-425b-4e6b-bc10-5b0333dc9ad1.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key=Qdrant_api_key,
    collection_name="my_collection",
    force_recreate=True # Add this line to recreate the collection if it doesn't exist
    )
    dense_retriever = qdrant_store.as_retriever(
        search_kwargs={
            "k": 8,                  # top-k
            "score_threshold": 0.25  # filter low scores
        }
    )
    bm25_retriever = BM25Retriever.from_documents(normal_chunks)
    bm25_retriever.k = 8
    hybrid_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, dense_retriever],
        weights=[0.4, 0.6]
    )
    return hybrid_retriever

# Agent

In [ ]:
def call_model(question,hybrid_retriever):
    qna_template = "\n".join(["""
              You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
              If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
              Question: {question}
              Context: {context}
              Answer:
    """

    ])

    qna_prompt = PromptTemplate(
        template=qna_template,
        input_variables=['context', 'question'],
        verbose=True
    )


    stuff_chain = create_stuff_documents_chain(llm, prompt=qna_prompt)
    question=question
    retreive_doc= hybrid_retriever.get_relevant_documents(question)
    answer = stuff_chain.invoke(
        {
            "context": retreive_doc,
            "question": question
        }
    )
    return answer

In [ ]:
chunk=prepare_data(url)

Token indices sequence length is longer than the specified maximum sequence length for this model (537 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
hybrid_retriever=Hybrid_search(chunk)

In [ ]:
model=call_model(question,hybrid_retriever)

In [ ]:
display(Markdown(model))

Here is the list of courses in the Main Specialization Requirements with their hours and grades:

**Compulsory Subjects (39 hours):**
- Artificial Intelligence (AI 310): 3 hours, Grade C+
- Image Processing-1 (IT 441): 3 hours, Grade B+
- Convex Optimization Theory (AI 320): 3 hours, Grade C+
- Evolutionary Algorithm (AI 420): 3 hours, Grade C
- Machine Learning (AI 330): 3 hours, Grade B+
- Computational Intelligence (AI 430): 3 hours, Grade B+
- Neural Networks & Deep Learning (AI 335): 3 hours, Grade A+
- Natural Language Processing (AI 360): 3 hours, Grade B
- Advanced Machine Learning (AI 370): 3 hours, Grade B+
- Big Data Technologies (IS 365): 3 hours, Grade C+
- Natural Language Understanding (AI 460): 3 hours, Grade A
- Project (AI 0): 6 hours, Grade A+

**Elective Subjects (21 hours):**
- Microprocessors (IT 312): 3 hours
- Cryptography (CS 480): 3 hours
- Parallel Processing and High Performance Computing (CS 471): 3 hours, Grade A
- Distributed Systems and Cloud Computing (CS 460): 3 hours, Grade A+
- Embedded Systems (IT 414): 3 hours
- Advanced Algorithms (CS 416): 3 hours
- Digital Signal Processing (IT 341): 3 hours
- Human Computer Interaction (AI 380): 3 hours
- Bioinformatics (AI 490): 3 hours
- Speech Processing (IT 443): 3 hours, Grade B
- Computer Vision (IT 444): 3 hours, Grade A+
- Introduction to Data Science (IS 360): 3 hours, Grade B+
- Knowledge Representation (AI 457): 3 hours
- Database Systems-2 (IS 312): 3 hours, Grade B
- Robotics (IT 415): 3 hours, Grade B
- Spoken Language Processing (AI 435): 3 hours
- Data Visualization (AI 350): 3 hours
- Embedded Internet of Things System (IT 425): 3 hours
- Selected Topics in Artificial Intelligence (AI 496): 3 hours